# POD-DL-ROM
This code prepares the data to be given to the POD-DL-ROM structure described in https://www.sciencedirect.com/science/article/pii/S0045782521005120. Read carefully the comments at the beginning of every block. Every block is numbered accordingly to the order it should be called. At the beginning you will have to choose whether to use blood clots data or generic data you want to use: follow direction a for blood clot or b for what you want. At the end the input and testing files will go to the path: ../MODELS/specie/interval/N/Data, where MODELS is a generic directory which contains all the different cases we want to analyze (species, number of modes), specie defines the biochemical specie we want to study, interval is used to separate the data when we train indipendently based on a cluster, N is the number of modes, Data is a directory which contains the inputs for the DL-ROM structure (coefficients and parameters), the POD basis V and the testing files.

# LIBRARIES AND FUNCTIONS

In [ ]:
#1
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.interpolate
import os
import copy
import pandas as pd
import csv
from scipy.interpolate import griddata
from numpy import savetxt
import sklearn.utils.extmath
import progressbar
from time import sleep

In [ ]:
#2
def get_field(path,simulations,seconds,mesh,field):
    Ns = len(simulations)*len(seconds)
    S = np.zeros((Ns,mesh))
    row = 0
    bar = progressbar.ProgressBar(maxval=len(simulations), \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    for i,s in enumerate(simulations):
        bar.update(i+1)
        for t in seconds:
            data = get_data(path+'/'+'sim_'+str(s)+'/'+str(t)+'/'+field,mesh)
            S[row,:] = data
            row+=1
        sleep(0.1)
    bar.finish()
    return S

def get_data(path,mesh):
    with open(path, 'r') as file:
        output = [None]*mesh
        count = 0
        start = False
        for line in file:
            #print(str(line))
            if line[0] == ')':
                break
            if start:
                output[count] = (float(line))
                #print(output[count])
                count +=1
            if line[0] == "(":
                start = True
        return np.array(output)

def read_files_centers(file_path):
    with open(file_path, 'r') as file:
        if file_path[-1] != 'U':
            lines = np.array(file.read().split("\n"))
            start = np.where(lines == "(")
            end = np.where(lines == ")")
            if len(start[0]) != 0:
                output = np.array(lines[start[0][0]+1:end[0][0]])
                return output
            else:
                return []
        else:
            U = read_U(file_path)
            return U

def M_matrix(simulations,time,parameters,path):
    file = open(path+'/X_LHS_Uniform.csv')
    csvreader = csv.reader(file)
    #header = next(csvreader)
    #print(header)
    rows = []
    for count,row in enumerate(csvreader):
        if count+1<=simulations:
            rows.append(row)
        else: 
            break
    file.close()
    M = np.zeros((simulations*len(time),parameters))#parameters = parameters+1
    count = 0
    for i in rows:
        for t in time:
            first = np.array([t])
            second = np.array(i)
            #print(count*time+t)
            M[count,:] = np.concatenate([first,second])
            count+=1
    return M

def normalize_M(M,path):
    M_norm = np.zeros((np.shape(M)[0],np.shape(M)[1]))
    M_max = np.amax(M, axis=0)
    M_min = -np.amax(-M,axis=0)
    df = pd.read_csv(path+'/normalization.csv') 
    df['max_M'] = M_max
    df.to_csv(path+'/normalization.csv',index=False)
    df['min_M'] = M_min
    df.to_csv(path+'/normalization.csv',index=False)
    for i in range(np.shape(M)[0]):
        for j in range(np.shape(M)[1]):
            M_norm[i,j] = (M[i,j]- M_min[j])/(M_max[j]-M_min[j])
    return M_norm

def normalize_S(S,path,n_params):
    S_max = np.amax(S)
    S_min = -np.amax(-S)
    maxi = [None]*n_params
    mini = [None]*n_params
    maxi[0] = S_max
    mini[0] = S_min
    maxmin = {'max_S':maxi,'min_S':mini}
    df = pd.DataFrame(maxmin,dtype = 'float')
    df.to_csv(path+'normalization.csv')
    return (S-S_min)/(S_max-S_min)
        
def inverse_normalize_S(s):
    S_max = np.amax(S)
    S_min = -np.amax(-S)
    return (S_max-S_min)*S+S_min

def inverse_normalize_M(M):
    M_norm = np.zeros((np.shape(M)[0],np.shape(M)[1]))
    M_max = np.amax(M, axis=0)
    M_min = -np.amax(-M,axis=0)
    for i in range(np.shape(M)[0]):
        for j in range(np.shape(M)[1]):
            M_norm[i,j] = (M_max[j]-M_min[j])*M[j,i]+M_min[j]
    return M_norm

def velocity_field(S,dt = 1):
    x,y = np.shape(S)
    new_S = np.zeros((x,y))
    for i in range(x):
        if i%40==0:
            new_S[i,:] = S[i,:]
        else:
            new_S[i,:] = (S[i,:]-S[i-1,:])/dt
            
    return new_S

def get_av_max_min_persec(S):
    arr_ma = np.zeros(40)
    arr_mi = np.zeros(40)
    for i in range(40):
        ma = 0
        mi = 0
        for j in range(95):
            ma = max(ma,np.amax(S[i+j*40,:]))
            mi = min(mi,-np.amax(-S[i+j*40,:]))
            
        arr_ma[i] = ma
        arr_mi[i] = mi
    return arr_ma,arr_mi
    
def normalize_bysec(S,ma,mi):
    x = np.shape(S)[0]
    y = np.shape(S)[1]
    S_norm = np.zeros((x,y))
    for i in range(x):
        S_norm[i,:] = (S[i,:]-mi[i%40])/(ma[i%40]-mi[i%40])
    return S_norm

def get_max_min_feature(S):
    mean = np.mean(S,axis = 0)
    std = -np.std(-S,axis = 0)
    return mean,std

def normalize_s_byfeature(S,mean,std):
    S_norm = np.zeros((np.shape(S)[0],np.shape(S)[1]))
    for i in range(np.shape(S)[0]):
        for j in range(np.shape(S)[1]):
            S_norm[i,j] = (S[i,j]-mean[j])/std[j]
    return S_norm


def padding(S,n):
    arr = np.zeros((np.shape(S)[0],n))
    S = np.hstack((S,arr))
    return S

def which_padding(length):
    count = 0
    while True:
        if (length+count)**0.5 ==int((length+count)**0.5):
            return count
        else:
            count+=1
            
def variance_simulations(S,t,seconds,sim):
    x,y = np.shape(S)
    var_mat = np.zeros((sim,y))
    for i in range(sim):
        var_mat[i,:] = S[(t-2)+i*seconds,:]   
    var_arr = np.std(var_mat,axis=0)
    ma = np.amax(var_arr)
    mi = np.amin(var_arr)
    return (var_arr-mi)/(ma-mi)
    
def KPOD(S,gamma,N):
    S = S.T
    x,y = np.shape(S)
    K = np.zeros((y,y))
    bar = progressbar.ProgressBar(maxval=y, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    for i in range(y):
        bar.update(i+1)
        for j in range(y):
            if j>i:
                K[i,j] = np.exp(-gamma*np.linalg.norm(S[:,i]-S[:,j])**2)
            elif j == i:
                K[i,j] = 1
            else:
                K[i,j] = K[j,i]
        sleep(0.1)
    bar.finish()
    l,v = np.linalg.eig(K)
    print('eig done')
    a = []
    for i in v:
        a.append(i.real.copy())
    P = np.zeros((x,y))
    bar = progressbar.ProgressBar(maxval=y, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    for i in range(y):
        bar.update(i+1)
        P[:,i] = 1/np.sqrt(l[i])*np.matmul(S,a[i])
        sleep(0.1)
    bar.finish()
    q,r = np.linalg.qr(P)
    return K,l,q,r

def formatNumber(num):
    arr = []
    for count,i in enumerate(num):
        if i % 1 == 0:
            arr.append(int(i))
        else:
            arr.append(i)
    return arr

def interpolate(x,y,z,step,method = 'cubic'):
    xi = np.arange(0,0.0006,step)
    yi = np.arange(0,0.0035,step)
    xi,yi = np.meshgrid(xi,yi)
    mask = ((xi > 0.0001) & (yi < 0.00118)) | ((xi > 7.9e-5) & (yi < 0.001784)&(yi > 0.001604))
    # interpolate
    zi = griddata((x,y),z,(xi,yi),method=method)
    zi[mask] = np.nan
    return xi,yi,zi

# ACQUISITION OF DATA
You can choose the direction a or direction b. In the first case you will take blood clots data, in the second one you will take whatever data you have already prepared. The matrices S and M are collected accordingly to the paper cited at the beginning. Notice that here we take the transposed of what is said in the paper, since it is easier to deal with them like this with python.

In [ ]:
#3a
#use this if you want blood clot simulations 
#choose exactly which simulations and which seconds from the simulations you want to get
which_sim = np.arange(1,101,1)
print('Simulation taken = '+str(which_sim))
print('Number of simulation taken = '  +str(len(which_sim)))
range_time = np.arange(2,42,1)
rang_time = formatNumber(range_time)
print('Time interval = '+str(range_time))
print('Length simulation = '  +str(len(range_time)))

#specify the biochemical specie, the number of POD modes N and in which interval you want to save the data
#the interval is used if you want to train indipendently for example three clusters. If you want to do all
#together just choose interval = 'full'. You also define the number of parameters used
specie = 'vWFs'
N = 64
interval = 'full'
n_params = 6 #including time

In [ ]:
#4a
#get data from file; be careful to the name of the file and to the directory where the simulations is saved

basic_data = '../'
S = (get_field(basic_data+'/DoE',which_sim,range_time,68650,specie))
M =(M_matrix(len(which_sim),range_time,6,basic_data))
cell_centers = read_files_centers(basic_data+'/cellCenters')
cell_true = []
for i in cell_centers:
    cell_true.append(np.array(i[1:-1].split()).astype(np.float64))
cell_true = np.array(cell_true)
print('Shape S = ' + str(np.shape(S)))
print('Shape M = '+ str(np.shape(M)))

In [ ]:
#5a
#create the necessary directories
path = '../MODELS/'+specie+'/'+interval+'/'+str(N)+'/'+'Data/'
os.makedirs(path,exist_ok=True)

In [ ]:
#3b
#use only if you have not used direction a!

#Define S and M as above
S = #you know it
M = #you know it 

In [ ]:
#4b #use only if you have not used direction a!

In [ ]:
#5b
#use only if you have not used direction a!

#create the necessary directories
path = 'whatever you want'
os.makedirs(path)
os.makedirs(path,exist_ok=True)

# PREPARE INPUT DATA AND COMPUTE POD
Now you do not have dependance on a or b

In [ ]:
#6

#divide S and M in train and test. Change the ratio training/test as you want.
S_train = copy.copy(S[0:3800])
S_test = copy.copy(S[-200:])
M_train = copy.copy(M[0:3800])
M_test = copy.copy(M[-200:])
print('S_train length = '+str(np.shape(S_train)))
print('S_test length = '+str(np.shape(S_test)))
print('M_train length = '+str(np.shape(M_train)))
print('S_test length = '+str(np.shape(M_test)))

In [ ]:
#7

#performs random SVD on S_train given N number of modes
_,s,v = sklearn.utils.extmath.randomized_svd(S_train,N,random_state = 8) #0.8597410118425659 0.98364044904060282

In [ ]:
#8

#define basis Matrix V and Input coefficients of the Encoder
V_transp = v
Input = np.zeros((np.shape(S_train)[0],N))
print('Shape of V = '+str(np.shape(V_transp)))
print('Shape of Input = '+str(np.shape(Input)))

In [ ]:
#9
#project the snapshot onto V in order to get the coefficients which will be the input of the Encoder

for count,data in enumerate(S_train):
    Input[count,:] = np.matmul(V_transp,data)
print('Shape of Input = '+str(np.shape(Input)))


In [ ]:
#10

# normalization as explained in the paper and save statistics of normalization to use them during testing

Input = normalize_S(Input,path,n_params)
M_norm = normalize_M(M_train,path)

In [ ]:
#11

#save training matrices

savetxt(path+'/Input.csv', Input, delimiter=',')
savetxt(path+'/M_train.csv',M_norm,delimiter = ',')
print('Shape of Input = '+str(np.shape(Input)))
print('Shape of M = '+str(np.shape(M_norm)))

In [ ]:
#12

#save test matrices

savetxt(path+'/S_test.csv', S_test, delimiter=',')
savetxt(path+'/M_test.csv', M_test, delimiter=',')
print('Shape of S_test = '+str(np.shape(S_test)))
print('Shape of M_test = '+str(np.shape(M_test)))

In [ ]:
#13

#save V matrix
savetxt(path+'/V.csv', v.transpose(), delimiter=',')
print('Shape of V = '+str(np.shape(v.transpose())))

#end acquisition of data. Below just some analysis, no need to run them but it's a good practice to 
#plot the input files to see if everything is as expected

# END OF ACQUISITION AND TREATMENT OF DATA

Now you can send Input and M_train to the cluster to run them. But before you should look at the data below to be sure they are correct

In [ ]:
#control variance over simulations
var_arr = variance_simulations(S,39,40,100)


In [ ]:
fig,ax = plt.subplots(figsize = (10,10))
cm = plt.cm.get_cmap('RdYlBu_r')
#ax = fig.gca(projection='3d')
scatter_plot = ax.scatter(cell_true[:,0], cell_true[:,1], c = var_arr, lw=0, s=1,cmap=cm)
plt.colorbar(scatter_plot)

In [ ]:
from scipy import interpolate
fun = interpolate.interp2d(cell_true[:,0], cell_true[:,1],S[498,:] , kind='linear')

In [ ]:
#check POD modes
t = 4
fig,ax = plt.subplots(1,4,figsize = (20,10))
cm = plt.cm.get_cmap('RdYlBu_r')
#ax = fig.gca(projection='3d')
scatter_plot = ax[0].scatter(cell_true[:,0], cell_true[:,1], c = S[498,:], lw=0, s=1,cmap=cm)
plt.colorbar(scatter_plot, ax=ax[0])
scatter_plot = ax[1].scatter(cell_true[:,0], cell_true[:,1], c = S2[498,:], lw=0, s=1,cmap=cm)
plt.colorbar(scatter_plot, ax=ax[1])
scatter_plot = ax[2].scatter(cell_true[:,0], cell_true[:,1], c = S[777,:], lw=0, s=1,cmap=cm)
plt.colorbar(scatter_plot, ax=ax[2])
scatter_plot = ax[3].scatter(cell_true[:,0], cell_true[:,1], c = S2[777,:], lw=0, s=1,cmap=cm)
plt.colorbar(scatter_plot, ax=ax[3])
plt.savefig('/people/longhi/Bureau/Useful_pic/firstmodes1-4_'+str(t)+'.png',dpi=150,facecolor='w',transparent=True)


In [ ]:
fig,ax = plt.subplots(1,4,figsize = (15,10))
cm = plt.cm.get_cmap('RdYlBu_r')
#ax = fig.gca(projection='3d')
scatter_plot = ax[0].scatter(cell_true[:,0], cell_true[:,1], c = v[4,:], lw=0, s=1,cmap=cm)
scatter_plot = ax[1].scatter(cell_true[:,0], cell_true[:,1], c = v[5,:], lw=0, s=1,cmap=cm)
scatter_plot = ax[2].scatter(cell_true[:,0], cell_true[:,1], c = v[6,:], lw=0, s=1,cmap=cm)
scatter_plot = ax[3].scatter(cell_true[:,0], cell_true[:,1], c = v[7,:], lw=0, s=1,cmap=cm)
plt.colorbar(scatter_plot)
plt.savefig('/people/longhi/Bureau/Useful_pic/first_modes5-8_'+str(t)+'.png',dpi=150,facecolor='w',transparent=True)

In [ ]:
fig,ax = plt.subplots(1,4,figsize = (15,10))
cm = plt.cm.get_cmap('RdYlBu_r')
#ax = fig.gca(projection='3d')
scatter_plot = ax[0].scatter(cell_true[:,0], cell_true[:,1], c = v[8,:], lw=0, s=20,cmap=cm)
scatter_plot = ax[1].scatter(cell_true[:,0], cell_true[:,1], c = v[9,:], lw=0, s=20,cmap=cm)
scatter_plot = ax[2].scatter(cell_true[:,0], cell_true[:,1], c = v[10,:], lw=0, s=20,cmap=cm)
scatter_plot = ax[3].scatter(cell_true[:,0], cell_true[:,1], c = v[11,:], lw=0, s=20,cmap=cm)
plt.colorbar(scatter_plot)
plt.savefig('/people/longhi/Bureau/Useful_pic/first_modes9-12_'+str(t)+'.png',dpi=150,facecolor='w',transparent=True)

In [ ]:
x1,x2,m1 = interpolate(cell_true[:,0],cell_true[:,1],v[0,:],1e-6)
x1,x2,S1 = interpolate(cell_true[:,0],cell_true[:,1],S_train[39,:],1e-6)
x1,x2,m2 = interpolate(cell_true[:,0],cell_true[:,1],v[1,:],1e-6)
x1,x2,S2 = interpolate(cell_true[:,0],cell_true[:,1],S_train[20,:],1e-6)
x1,x2,m3 = interpolate(cell_true[:,0],cell_true[:,1],v[142,:],1e-6)
x1,x2,S3 = interpolate(cell_true[:,0],cell_true[:,1],S_train[0,:],1e-6)

In [ ]:
fig,ax = plt.subplots(2,3,figsize = (15,10))
cm = plt.cm.get_cmap('jet')
#ax = fig.gca(projection='3d')
scatter_plot = ax[0][0].scatter(x1, x2, c = m1, lw=0, s=0.5,cmap=cm)
plt.colorbar(scatter_plot, ax=ax[0][0], label = 'nmol $m^{-3}$')
ax[0][0].axis('off')
ax[0][0].set_title('Mode number 1')
scatter_plot = ax[1][0].scatter(x1, x2, c = S1, lw=0, s=0.5,cmap=cm)
plt.colorbar(scatter_plot, ax=ax[1][0], label = 'nmol $m^{-3}$')
ax[1][0].axis('off')   
ax[1][0].set_title('Snapshot, t = 41 s')
scatter_plot = ax[0][1].scatter(x1, x2, c = m2, lw=0, s=0.5,cmap=cm)
plt.colorbar(scatter_plot, ax=ax[0][1], label = 'nmol $m^{-3}$')
ax[0][1].axis('off')
ax[0][1].set_title('Mode number 2')
scatter_plot = ax[1][1].scatter(x1, x2, c = S2, lw=0, s=0.5,cmap=cm)
plt.colorbar(scatter_plot, ax=ax[1][1], label = 'nmol $m^{-3}$')
ax[1][1].axis('off')
ax[1][1].set_title('Snapshot, t = 22 s')
scatter_plot = ax[0][2].scatter(x1, x2, c = m3, lw=0, s=0.5,cmap=cm)
plt.colorbar(scatter_plot, ax=ax[0][2], label = 'nmol $m^{-3}$')
ax[0][2].axis('off')
ax[0][2].set_title('Mode number 142')
scatter_plot = ax[1][2].scatter(x1, x2, c = S3, lw=0, s=0.5,cmap=cm)
plt.colorbar(scatter_plot, ax=ax[1][2], label = 'nmol $m^{-3}$')
ax[1][2].axis('off')
ax[1][2].set_title('Snapshot, t = 2 s')
plt.savefig('modes_snapshot.png',dpi=200,facecolor='w',transparent=True,bbox_inches='tight')

In [ ]:
a = []
num = 0
for count,i in enumerate(v1):
    scal = np.matmul(i,S_train[num,:])/(np.linalg.norm(i)*np.linalg.norm(S_train[num,:]))
    a.append(scal)
    if np.abs(scal) > 0.2:
        print(scal)
        print(count)
        print('next')
    
plt.figure()
plt.plot(a,'*')
#plt.xlim([0,300])

In [ ]:
fig,ax = plt.subplots(1,2,figsize = (15,10))
cm = plt.cm.get_cmap('RdYlBu_r')
#ax = fig.gca(projection='3d')
scatter_plot = ax[0].scatter(cell_true[:,0], cell_true[:,1], c = S[0,:], s=1,cmap=cm)
plt.colorbar(scatter_plot, ax=ax[0])
scatter_plot = ax[1].scatter(cell_true[:,0], cell_true[:,1], c = v[0,:], s=1,cmap=cm)
plt.colorbar(scatter_plot, ax=ax[1])

In [ ]:
B = np.matmul(np.diag(s),v)
np.shape(v)

In [ ]:
a = []
num = 0
for count,i in enumerate(B):
    scal = np.matmul(i,S_train[num,:])/(np.linalg.norm(i)*np.linalg.norm(S_train[num,:]))
    a.append(scal)
    if scal > 0.2:
        print(scal)
        print(count)
        print('next')
    
plt.figure()
plt.plot(a)

#plt.xlim([0,500])

In [ ]:
fig,ax = plt.subplots(1,4,figsize = (15,10))
cm = plt.cm.get_cmap('RdYlBu_r')
#ax = fig.gca(projection='3d')
scatter_plot = ax[0].scatter(cell_true[:,0], cell_true[:,1], c = v[:,0], lw=0, s=20,cmap=cm)
scatter_plot = ax[1].scatter(cell_true[:,0], cell_true[:,1], c = v[:,1], lw=0, s=20,cmap=cm)
scatter_plot = ax[2].scatter(cell_true[:,0], cell_true[:,1], c = v[:,2], lw=0, s=20,cmap=cm)
scatter_plot = ax[3].scatter(cell_true[:,0], cell_true[:,1], c = v[:,3], lw=0, s=20,cmap=cm)
plt.colorbar(scatter_plot)

In [ ]:
q,r = KPOD(S,1e-5,256)

In [ ]:
fig,ax = plt.subplots(1,4,figsize = (15,10))
cm = plt.cm.get_cmap('RdYlBu_r')
#ax = fig.gca(projection='3d')
scatter_plot = ax[0].scatter(cell_true[:,0], cell_true[:,1], c = q[:,0], lw=0, s=20,cmap=cm)
scatter_plot = ax[1].scatter(cell_true[:,0], cell_true[:,1], c = q[:,1], lw=0, s=20,cmap=cm)
scatter_plot = ax[2].scatter(cell_true[:,0], cell_true[:,1], c = q[:,2], lw=0, s=20,cmap=cm)
scatter_plot = ax[3].scatter(cell_true[:,0], cell_true[:,1], c = q[:,3], lw=0, s=20,cmap=cm)
plt.colorbar(scatter_plot)

In [ ]:
c = []
num = 20
for count,i in enumerate(v2[0:256]):
    scal = np.matmul(i,S_test[num,:])/(np.linalg.norm(i)*np.linalg.norm(S_test[num,:]))
    c.append(scal)
    if np.abs(scal) > 0.2:
        print(scal)
        print(count)
        print('next')
    
plt.figure()
plt.plot(c)

#plt.xlim([0,500])

In [ ]:
fig,ax = plt.subplots(1,2,figsize = (17,8))
ax[0].plot(np.arange(1,257,1),s*s/np.sum(s*s),'*')
ax[0].set_yscale('log')
ax[0].set_title('Singular values of AP',fontsize=16)
ax[0].set_xlabel('Mode',fontsize=14)
ax[0].set_ylabel('Energy',fontsize=14)
ax[1].plot(np.arange(1,257,1),s2*s2/np.sum(s2*s2),'*')
ax[1].set_yscale('log')
ax[1].set_title('Singular values of vWFs',fontsize=16)
ax[1].set_xlabel('Mode',fontsize=14)
ax[1].set_ylabel('Energy',fontsize=14)
plt.savefig('singularvalue.png',dpi=150,facecolor='w',transparent=True,bbox_inches='tight' )
print(s*s/np.sum(s*s))

In [ ]:
ap = 0
vwfs = 0
for i in range(256):
    if (s*s)[i]/np.sum(s*s)>10**(-3):
        ap+=1
    if (s2*s2)[i]/np.sum(s2*s2)>10**(-3):
        vwfs+=1
print(ap) 
print(vwfs)

In [ ]:
x1,x2,_ = interpolate(cell_true[:,0],cell_true[:,1],S[0,:],1e-6)

In [ ]:
_,_,S1 = interpolate(cell_true[:,0],cell_true[:,1],S[5,:],1e-6)
_,_,S2 = interpolate(cell_true[:,0],cell_true[:,1],S[20,:],1e-6)
_,_,S3 = interpolate(cell_true[:,0],cell_true[:,1],S[39,:],1e-6)

In [ ]:
fig,ax = plt.subplots(1,3,figsize = (15,10))
cm = plt.cm.get_cmap('jet')
scatter_plot = ax[0].scatter(x1, x2, c = S1, lw=0, s=0.5,cmap=cm,vmin = 0)
ax[0].set_title('t = 7 s', fontsize = 14)
ax[0].axis('off')
plt.colorbar(scatter_plot, ax=ax[0], label = 'nmol $m^{-3}$')
scatter_plot = ax[1].scatter(x1, x2, c = S2, lw=0, s=0.5,cmap=cm,vmin = 0)
ax[1].set_title('t = 22 s', fontsize = 14)
ax[1].axis('off')
plt.colorbar(scatter_plot, ax=ax[1],label = 'nmol $m^{-3}$')
scatter_plot = ax[2].scatter(x1, x2, c = S3, lw=0, s=0.5,cmap=cm,vmin = 0)
ax[2].set_title('t = 41 s', fontsize = 14)
ax[2].axis('off')
plt.colorbar(scatter_plot, ax=ax[2],label = 'nmol $m^{-3}$')
plt.savefig('AP.png',dpi=200,facecolor='w',transparent=True,bbox_inches='tight')

In [ ]:
r,v,m = var_arr4 = variance_simulations(S,41,40,100)
fig,ax = plt.subplots(1,3,figsize = (15,10))
cm = plt.cm.get_cmap('RdYlBu_r')
#ax = fig.gca(projection='3d')
scatter_plot = ax[0].scatter(cell_true[:,0], cell_true[:,1], c = m, lw=0, s=20,cmap=cm)
plt.colorbar(scatter_plot, ax=ax[0])
scatter_plot = ax[1].scatter(cell_true[:,0], cell_true[:,1], c = v, lw=0, s=20,cmap=cm)
plt.colorbar(scatter_plot, ax=ax[1])
scatter_plot = ax[2].scatter(cell_true[:,0], cell_true[:,1], c = r, lw=0, s=1,cmap=cm)
plt.colorbar(scatter_plot, ax=ax[2])

In [ ]:
print(np.where(var_arr1>1))
print(m[591])
print(v[591])
print(v[591]/m[591])

In [ ]:
#control variance over simulations
var_arr1 = variance_simulations(S,10,40,100)
var_arr4 = variance_simulations(S,41,40,100)

In [ ]:
#interpolate normalized variances
x1,x2,norm_var1 = interpolate(cell_true[:,0],cell_true[:,1],var_arr1,1e-6)
x1,x2,S1 = interpolate(cell_true[:,0],cell_true[:,1],S[8,:],1e-6)
x1,x2,norm_var2 = interpolate(cell_true[:,0],cell_true[:,1],var_arr4,1e-6)
x1,x2,S2 = interpolate(cell_true[:,0],cell_true[:,1],S[39,:],1e-6)

In [ ]:
fig,ax = plt.subplots(figsize = (10,10))
cm = plt.cm.get_cmap('RdYlBu_r')
#ax = fig.gca(projection=‘3d’)
scatter_plot = ax.scatter(cell_true[:,0], cell_true[:,1], c = var_arr1, lw=0, s=1,cmap=cm, vmax = 0.3)
plt.colorbar(scatter_plot)

In [ ]:
fig,ax = plt.subplots(2,2,figsize = (15,10))
cm = plt.cm.get_cmap('jet')
#ax = fig.gca(projection=‘3d’)
scatter_plot = ax[0][0].scatter(x1, x2, c = norm_var1, lw=0, s=0.5,cmap=cm,vmin = 0)
ax[0][0].set_title('Normalized Standard Deviation, t = 12 s', fontsize = 14)
ax[0][0].axis('off')
plt.colorbar(scatter_plot, ax=ax[0][0])
scatter_plot = ax[0][1].scatter(x1, x2, c = S1, lw=0, s=0.5,cmap=cm,vmin = 0)
ax[0][1].set_title('Snapshot, t = 12 s', fontsize = 14)
ax[0][1].axis('off')
plt.colorbar(scatter_plot, ax=ax[0][1], label = 'nmol $m^{-3}$')
scatter_plot = ax[1][0].scatter(x1, x2, c = norm_var2, lw=0, s=0.5,cmap=cm,vmin = 0)
ax[1][0].set_title('Normalized Standard Deviation, t = 41 s', fontsize = 14)
ax[1][0].axis('off')
plt.colorbar(scatter_plot, ax=ax[1][0])
scatter_plot = ax[1][1].scatter(x1, x2, c = S2, lw=0, s=0.5,cmap=cm,vmin = 0)
ax[1][1].set_title('Snapshot, t = 41 s', fontsize = 14)
ax[1][1].axis('off')
plt.colorbar(scatter_plot, ax=ax[1][1], label = 'nmol $m^{-3}$')
plt.savefig('vWFs_std_normalized.png',dpi=200,facecolor='w',transparent=True,bbox_inches='tight')

In [ ]:
x1,x2,m1 = interpolate(cell_true[:,0],cell_true[:,1],S[37,:],1e-6)
fig,ax = plt.subplots(1,1,figsize = (5,7))
cm = plt.cm.get_cmap('jet')
scatter_plot = ax.scatter(x1, x2, c = m1, lw=0, s=0.5,cmap=cm)
plt.colorbar(scatter_plot, label = 'nmol $m^{-3}$')
ax.axis('off')
ax.set_title('vWFs concentration',fontsize = 14)
plt.savefig('FOM.png',dpi=200,facecolor='w',transparent=True,bbox_inches='tight')

In [ ]:
X = Input
fcm = FCM(n_clusters=2)
fcm.fit(X)

In [ ]:
# outputs
fcm_centers = fcm.centers
fcm_labels = fcm.predict(X)

# plot result
f, axes = plt.subplots(1, 2, figsize=(11,5))
axes[0].scatter(X[:,0], X[:,1], alpha=.5)
axes[1].scatter(X[:,0], X[:,1], c=fcm_labels, alpha=.5)
axes[1].scatter(fcm_centers[:,0], fcm_centers[:,1], marker="+", s=500, c='b')
plt.savefig('cluster.png',dpi=200,facecolor='w',transparent=True)
plt.show()

In [ ]:
plt.plot(fcm_labels[0:120],'o')


In [ ]:
fig,ax = plt.subplots(2,2,figsize = (10,10))
cm = plt.cm.get_cmap('RdYlBu_r')
#ax = fig.gca(projection=‘3d’)
scatter_plot = ax[0][0].scatter(cell_true[:,0], cell_true[:,1], c = S[0,:], lw=0, s=0.5,cmap=cm,vmin = 0)
ax[0][0].set_title('t = 13 s', fontsize = 14)
ax[0][0].axis('off')
plt.colorbar(scatter_plot, ax=ax[0][0])
scatter_plot = ax[0][1].scatter(cell_true[:,0], cell_true[:,1], c = S[1,:], lw=0, s=0.5,cmap=cm,vmin = 0)
ax[0][1].set_title('t = 14 s', fontsize = 14)
ax[0][1].axis('off')
plt.colorbar(scatter_plot,ax=ax[0][1])
scatter_plot = ax[1][0].scatter(cell_true[:,0], cell_true[:,1], c = S[2,:], lw=0, s=0.5,cmap=cm,vmin = 0)
ax[1][0].set_title('t = 15 s', fontsize = 14)
ax[1][0].axis('off')
plt.colorbar(scatter_plot, ax=ax[1][0])
scatter_plot = ax[1][1].scatter(cell_true[:,0], cell_true[:,1], c = S[3,:], lw=0, s=0.5,cmap=cm,vmin = 0)
ax[1][1].set_title('t = 16 s', fontsize = 14)
ax[1][1].axis('off')
plt.colorbar(scatter_plot,ax=ax[1][1])
plt.savefig('fig',dpi=150,facecolor='w',transparent=True,bbox_inches='tight')